# Reciprocal Relationship between Bartering and Selling
- When it comes to game exchange, do people prefer to exchange the game with someone who has exchanged before or someone who has not exchanged before? 
- What's the difference between game sales and game exchange?

Here, we use logistic regression.

## 0. Load the data

In [2]:
import pandas as pd
import pickle

In [3]:
# Load the data from pickle file
with open('data/transaction_history_all.pickle', 'rb') as f:
    data = pickle.load(f)

# Choose only the data before 2019-01-01
data = data.loc[data['created_utc']<='2018-12-31']

In [4]:
# with open(("./data/df_pre2019_original.pickle"), 'rb') as handle:
#     data = pickle.load(handle)
# data.shape

## 1. Preprocessing
- The bot may record wrongly the confirmantion.
- Some Reddit users may delete their accounts.

**We ignore these cases by deleting these rows.**

In [5]:
data.loc[data['author']==data['reply_author']].shape

(97, 11)

In [6]:
data = data.loc[data['author']!=data['reply_author']]

In [7]:
data.loc[(data['author']=='[deleted]')|(data['reply_author']=='[deleted]')].shape

(1759, 11)

In [8]:
data = data.loc[(data['author']!='[deleted]') & (data['reply_author']!='[deleted]')]

In [9]:
data.head()

,user,index,author,body,created_utc,permalink,reply_author,reply_body,type,pred,subreddit
6,rdmentalist,rdmentalist_2,MrAce2C,Successful trade with u/rdmentalist. Gave Gear...,2016-10-23 21:59:35,/r/gameswap/comments/543ojp/completed_trade_th...,rdmentalist,Confirmed.,gameswap,NaN,gameswap
9,rdmentalist,rdmentalist_5,TradlyGent,swapped a 20% PSN code for Gears of War 1 and ...,2018-07-03 17:19:14,/r/gameswap/comments/7q4pkv/completed_swap_thr...,rdmentalist,Confirmed.,gameswap,NaN,gameswap
10,rdmentalist,rdmentalist_6,keokidream,Successful trade with /u/rdmentalist\n\nCall o...,2016-06-17 14:54:23,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed! thanks,gameswap,NaN,gameswap
11,rdmentalist,rdmentalist_7,testicles_two,"Traded Gears of War 1,2,3 and Judgement for $1...",2016-06-23 23:55:01,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed ! thanks,gameswap,NaN,gameswap
12,rdmentalist,rdmentalist_8,walterpwn,Successful trade with [/u/rdmentalist](https:/...,2016-05-12 19:43:00,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed :),gameswap,NaN,gameswap


In [10]:
# So we still have 28586 rows of confirmantion data.
data.shape

(28586, 11)

In [11]:
data['created_utc'].describe(datetime_is_numeric=True)

count                            28586
mean     2016-10-29 16:11:53.239487744
min                2013-10-01 19:51:32
25%                2015-08-04 21:05:31
50%                2017-01-23 03:03:51
75%                2018-02-25 17:14:02
max                2018-12-30 23:12:21
Name: created_utc, dtype: object

## 2. Relationship

In [12]:
data_sale = data.loc[data['type']=='gamesale']
data_swap = data.loc[data['type']=='gameswap']

In [13]:
swap_user = set(data_swap['author'])
sale_user = set(data_sale['author'])
both_user = swap_user.intersection(sale_user)

In [14]:
print("Active users in r/GameSwap:", len(swap_user))
print("Active users in r/GameSale:", len(sale_user))
print("Active users in both communities:", len(both_user))

Active users in r/GameSwap: 1492
Active users in r/GameSale: 2885
Active users in both communities: 452


In [15]:
data.head()

,user,index,author,body,created_utc,permalink,reply_author,reply_body,type,pred,subreddit
6,rdmentalist,rdmentalist_2,MrAce2C,Successful trade with u/rdmentalist. Gave Gear...,2016-10-23 21:59:35,/r/gameswap/comments/543ojp/completed_trade_th...,rdmentalist,Confirmed.,gameswap,NaN,gameswap
9,rdmentalist,rdmentalist_5,TradlyGent,swapped a 20% PSN code for Gears of War 1 and ...,2018-07-03 17:19:14,/r/gameswap/comments/7q4pkv/completed_swap_thr...,rdmentalist,Confirmed.,gameswap,NaN,gameswap
10,rdmentalist,rdmentalist_6,keokidream,Successful trade with /u/rdmentalist\n\nCall o...,2016-06-17 14:54:23,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed! thanks,gameswap,NaN,gameswap
11,rdmentalist,rdmentalist_7,testicles_two,"Traded Gears of War 1,2,3 and Judgement for $1...",2016-06-23 23:55:01,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed ! thanks,gameswap,NaN,gameswap
12,rdmentalist,rdmentalist_8,walterpwn,Successful trade with [/u/rdmentalist](https:/...,2016-05-12 19:43:00,/r/gameswap/comments/4c351t/completed_trade_th...,rdmentalist,Confirmed :),gameswap,NaN,gameswap


In [16]:
# Get the months in created_utc
data['month']=data['created_utc'].dt.to_period('M')
months = data['month'].sort_values().unique()


In [17]:
# Get the number of transactions in each month.
# So we can see if two users have ever interacted in the given month.
transaction_at_month = data.groupby(['author','reply_author','type','month']).size().reset_index(name='count')

In [41]:
interactions = transaction_at_month.pivot(index=['author','reply_author','month'], columns='type', values='count').fillna(0).astype(int)
interactions = interactions.reset_index().rename_axis(None, axis=1)
interactions['total'] = interactions['gamesale'] + interactions['gameswap']
interactions.head()

,author,reply_author,month,gamesale,gameswap,total
0,--El_Duderino--,Battlehead,2018-01,0,2,2
1,--El_Duderino--,CBUSraver,2016-09,0,2,2
2,--El_Duderino--,ComedyChef,2016-05,0,2,2
3,--El_Duderino--,FriendToGiraffes,2017-03,0,2,2
4,--El_Duderino--,HowelPendragon,2016-11,2,0,2


In [42]:
cumulative_transactions = {}
for month in months:
    cumulative_transactions[month] = data.loc[data['month']<=month].groupby(['user','type']).size().reset_index(name='count').pivot(index='user', columns='type', values='count').fillna(0).astype(int)
    if 'gamesale' not in cumulative_transactions[month].columns:
        cumulative_transactions[month]['gamesale']=0
    cumulative_transactions[month]['total'] = cumulative_transactions[month]['gamesale']+cumulative_transactions[month]['gameswap']
    cumulative_transactions[month] = cumulative_transactions[month].reset_index().rename_axis(None, axis=1)
    

In [43]:
cumulative_transactions[months[-1]] # The last month

,user,gamesale,gameswap,total
0,--el_duderino--,11,24,35
1,-arvin,1,0,1
2,-cdg-luck,0,1,1
3,-dark_link-,1,0,1
4,-elite-,1,0,1
...,...,...,...,...
8125,zyltis,0,2,2
8126,zyntakou,0,1,1
8127,zyx481,1,0,1
8128,zzgambit,3,0,3


### 2.1 Get transactions tuple
For each month $t$, if in month $t+1$, users $A$ and $B$ have an transaction, then we store tuple $(A_t, B_t, t)$, where $A_t$ is the number of cumulative transactions of user $A$ till month $t$.

In [40]:
for pre_month, current_month in zip(months[::2], months[1::2]):
    # 


SyntaxError: incomplete input (2954166287.py, line 2)